In [1]:
import numpy
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import string

In [2]:
import nltk
nltk.download("stopwords")

stopingWords=stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df=pd.read_csv("train.csv")

In [4]:
df.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


In [5]:
df.shape

(20800, 5)

In [6]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
df=df.fillna("")

In [8]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [9]:
df["content"]=df["title"]+" "+df["author"]+" "+df["text"]

In [10]:
df.duplicated().sum()

0

In [11]:
df=df.drop(["title", "author", "text"], axis=1)

In [12]:
df.head()

,id,label,content
0,0,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,2,1,Why the Truth Might Get You Fired Consortiumne...
3,3,1,15 Civilians Killed In Single US Airstrike Hav...
4,4,1,Iranian woman jailed for fictional unpublished...


In [13]:
ps=PorterStemmer()

def stemming(text):
    text=text.lower()
    a=text.split(" ")
    p=[]
    for i in a:
        i="".join([j for j in i if j not in string.punctuation])
        if i not in string.punctuation and i not in stopingWords:
            p.append(ps.stem(i))
    return  " ".join(p)


In [14]:
df["content"]=df["content"].apply(stemming)

In [15]:
df["content"]

0        hous dem aid didn’t even see comey’ letter jas...
1        flynn hillari clinton big woman campu breitbar...
2        truth might get fire consortiumnewscom truth m...
3        15 civilian kill singl us airstrik identifi je...
4        iranian woman jail fiction unpublish stori wom...
                               ...                        
20795    rapper ti trump ’poster child white supremacy’...
20796    nfl playoff schedul matchup odd new york time ...
20797    macy’ said receiv takeov approach hudson’ bay ...
20798    nato russia hold parallel exercis balkan alex ...
20799    keep f35 aliv david swanson david swanson auth...
Name: content, Length: 20800, dtype: object

In [16]:
df.head()

,id,label,content
0,0,1,hous dem aid didn’t even see comey’ letter jas...
1,1,0,flynn hillari clinton big woman campu breitbar...
2,2,1,truth might get fire consortiumnewscom truth m...
3,3,1,15 civilian kill singl us airstrik identifi je...
4,4,1,iranian woman jail fiction unpublish stori wom...


In [17]:
x=df["content"].values
y=df["label"].values

In [18]:
print(x[0])

hous dem aid didn’t even see comey’ letter jason chaffetz tweet darrel lucu hous dem aid didn’t even see comey’ letter jason chaffetz tweet darrel lucu octob 30 2016 subscrib jason chaffetz stump american fork utah imag courtesi michael jolley avail creativ commonsbi licens 
with apolog keith olbermann doubt worst person world week–fbi director jame comey accord hous democrat aid look like also know secondworst person well turn comey sent nowinfam letter announc fbi look email may relat hillari clinton’ email server rank democrat relev committe didn’t hear comey found via tweet one republican committe chairmen 
a know comey notifi republican chairmen democrat rank member hous intellig judiciari oversight committe agenc review email recent discov order see contain classifi inform long letter went oversight committe chairman jason chaffetz set polit world ablaz tweet fbi dir inform fbi learn exist email appear pertin investig case reopen 
— jason chaffetz jasoninthehous octob 28 2016 
of

In [19]:
y

array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

In [20]:
vectorizer=TfidfVectorizer()
vectorizer.fit(x)

x=vectorizer.transform(x)

In [21]:
print(x.shape)

(20800, 180081)


In [22]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test=train_test_split(x,y, test_size=0.2, random_state=0)

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [24]:
pipe=[
    {
        "name":"Logistic Regression",
        "model":LogisticRegression()
    },
       {
        "name":"KNeighborsClassifier",
       "model":KNeighborsClassifier(n_neighbors=5)
    },
       {
        "name":"SVC",
       "model":SVC(kernel="rbf")
    },
       {
        "name":"naive_bayes",
        "model":GaussianNB()
    },
       {
        "name":"Decision Tree",
        "model":DecisionTreeClassifier()
    },
       {
        "name":"RandomForest",
        "model":RandomForestClassifier()
    },
]


In [ ]:
from sklearn.metrics import confusion_matrix , accuracy_score

for i in pipe:
    print(i["name"], end=":")
    regressor=i["model"]
    regressor.fit(x_train, y_train)
    y_pred=regressor.predict(x_test)
    print(accuracy_score(y_test, y_pred))
    
    

Logistic Regression:0.9651442307692307
KNeighborsClassifier:0.8576923076923076
SVC: